# <span style='color:gold;'>Task: Synthesize and Analyze an "Impossible Chord"

In this assignment, you will create a musically impossible chord—one that cannot be physically played by humans or produced by any single instrument—using audio signal processing techniques. 

You will then analyze its properties to understand why it is impossible and how digital synthesis overcomes these limitations.

## Objective
Generate a chord spanning 8 `octaves` $$(C0 + C4 + C8)$$ and analyze its time-domain, frequency-domain, and perceptual characteristics to explain why it cannot be played by humans or traditional instruments.

## Steps to Complete

1. **Define the Impossible Chord**  
   - Identify the three constituent notes: C0, C4, and C8.  
   - Research and record their frequencies (use the `get_note_frequencies()` function for accuracy).  
   - Explain *why* this combination is impossible:  
     - Physical limitations of human hands and instruments  
     - Perceptual challenges with extreme frequencies  
     - Instrument range constraints  

2. **Generate Individual Note Signals**  
   - Use a 2-second duration and standard sample rate.  
   - Create an ADSR envelope for each note (attack=0.05s, decay=0.2s, sustain_level=0.7, release=0.5s) to simulate natural instrument behavior.  
   - Generate signals for each note using a piano timbre (via `generate_timbre()`).  

3. **Combine the Notes Safely**  
   - Use dynamic mixing to combine C4 (as the "melody") with C0 + C8 (as the "chord") to avoid clipping.  
   - Apply a 0.3-second fadeout to eliminate clicks at the end of the signal.  

4. **Visualize the Time Domain**  
   - Plot a 0.1-second segment of the combined chord.  
   - Describe the waveform: How do the three frequencies interact visually? Why is the waveform more complex than a single note?  

5. **Analyze Frequency Components**  
   - Perform an FFT to identify the dominant frequencies in the chord.  
   - Generate a spectrogram to visualize how frequency content changes over time.  
   - Verify that all three target frequencies (C0, C4, C8) appear in the analysis.  

6. **Evaluate Perception**  
   - Play the synthesized chord.  
   - Describe what you hear: Which notes are most/least prominent? Why?  
   - Connect your observations to the physical limitations of human hearing (e.g., sensitivity to mid-range frequencies vs. extreme lows/highs).  

## Deliverables
- A Jupyter notebook with your code (modified from the provided template as needed).  
- Written explanations for each step (addressing the questions above).  
- Plots of the time-domain waveform, FFT results, and spectrogram.  
- A 1-2 paragraph conclusion summarizing why this chord is impossible to play naturally and how digital synthesis enables its creation.  

## Deliverables
- A single html file with all code organized into logical steps and substeps.  
- The script must include comments explaining key operations.  

## Deadline
11pm this coming Sunday

## Grading Criteria
- Accuracy of signal generation (correct frequencies, envelopes, and mixing).  
- Quality of visualizations (clarity, proper labeling, relevant time/frequency ranges).  
- Depth of analysis (ability to connect technical properties to musical/physical limitations).  
- Correct use of audio processing functions (envelopes, mixing, fadeouts).  

Use the provided code template as a starting point, but add your own comments and explanations to demonstrate your understanding of the concepts.

# Step 1: 定义不可能的和弦与频率
本节将定义C0、C4、C8三个音符，并解释为何这种和弦无法被人类或传统乐器演奏。

In [ ]:
# Step 1: 定义C0、C4、C8的频率，并解释不可能的原因
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import sounddevice as sd
def get_note_frequencies():
    # 国际标准音高频率
    notes = {'C0': 16.35, 'C4': 261.63, 'C8': 4186.01}
    return notes
notes = get_note_frequencies()
print('C0频率:', notes['C0'], 'Hz')
print('C4频率:', notes['C4'], 'Hz')
print('C8频率:', notes['C8'], 'Hz')
# 物理/生理原因解释见markdown

# Step 2: 生成单个音符信号（含ADSR包络和钢琴音色）
本节将为每个音符生成2秒的信号，使用ADSR包络和钢琴音色。

In [ ]:
# Step 2: 生成单个音符信号
sr = 44100  # 采样率
duration = 2.0  # 秒
def adsr_envelope(length, sr, attack=0.05, decay=0.2, sustain_level=0.7, release=0.5):
    env = np.zeros(length)
    a = int(attack * sr)
    d = int(decay * sr)
    s = int((length/sr - attack - decay - release) * sr)
    r = int(release * sr)
    # Attack
    env[:a] = np.linspace(0, 1, a)
    # Decay
    env[a:a+d] = np.linspace(1, sustain_level, d)
    # Sustain
    env[a+d:a+d+s] = sustain_level
    # Release
    env[a+d+s:] = np.linspace(sustain_level, 0, r)
    return env
def generate_timbre(freq, sr, duration):
    t = np.linspace(0, duration, int(sr*duration), endpoint=False)
    # 简单钢琴音色：基频+谐波叠加
    signal = np.sin(2*np.pi*freq*t) + 0.5*np.sin(2*np.pi*2*freq*t) + 0.3*np.sin(2*np.pi*3*freq*t)
    env = adsr_envelope(len(signal), sr)
    return signal * env
note_signals = {}
for name, freq in notes.items():
    note_signals[name] = generate_timbre(freq, sr, duration)
    print(f'{name}音符信号已生成')

# Step 3: 混合音符并处理淡出
本节将动态混合C4（主旋律）与C0、C8（和弦），并在结尾加淡出处理，避免杂音。

In [ ]:
# Step 3: 混合音符并处理淡出
# 动态混合，避免溢出
chord = (note_signals['C0'] + note_signals['C8']) / 2
melody = note_signals['C4']
mix = melody * 0.6 + chord * 0.4
# 淡出处理，消除结尾杂音
fadeout_len = int(0.3 * sr)
fadeout = np.linspace(1, 0, fadeout_len)
mix[-fadeout_len:] *= fadeout
mix = mix / np.max(np.abs(mix))  # 归一化
print('和弦混合与淡出处理完成')

# Step 4: 时域波形可视化
本节将绘制和弦信号的0.1秒片段，观察多频率叠加后的波形复杂性。

In [ ]:
# Step 4: 时域波形可视化
plt.figure(figsize=(10,4))
segment = mix[:int(0.1*sr)]
plt.plot(segment)
plt.title('和弦信号的0.1秒时域波形')
plt.xlabel('样本点')
plt.ylabel('幅值')
plt.grid(True)
plt.show()
# 注释：可见多频率叠加后波形复杂

# Step 5: 频域分析（FFT与声谱图）
本节将分析和弦的主频成分，并绘制声谱图，验证C0、C4、C8频率均被合成。

In [ ]:
# Step 5: 频域分析（FFT与声谱图）
# FFT分析
fft = np.fft.rfft(mix)
freqs = np.fft.rfftfreq(len(mix), 1/sr)
plt.figure(figsize=(10,4))
plt.plot(freqs, np.abs(fft))
plt.title('和弦信号的频谱（FFT）')
plt.xlabel('频率 (Hz)')
plt.ylabel('幅值')
plt.xlim(0, 5000)
plt.grid(True)
plt.show()
# 声谱图
plt.figure(figsize=(10,4))
plt.specgram(mix, NFFT=2048, Fs=sr, noverlap=1024, cmap='plasma')
plt.title('和弦信号的声谱图')
plt.xlabel('时间 (秒)')
plt.ylabel('频率 (Hz)')
plt.colorbar(label='强度')
plt.ylim(0, 5000)
plt.show()
# 注释：可见C0、C4、C8频率均被合成

# Step 6: 播放和弦并听觉评价
本节将播放合成和弦，并描述听感。

In [ ]:
# Step 6: 播放和弦并听觉评价
sd.play(mix, sr)
sd.wait()
# 注释：中频C4最明显，极低C0和极高C8较难分辨，体现人耳频率敏感性

# Step 7: 总结与心得
本节总结为何该和弦无法自然演奏，以及数字合成的优势。